In [1]:
import numpy as np
import pandas as pd
import pickle
from urllib.parse import urlparse
from nltk.corpus import stopwords


from imblearn.over_sampling import RandomOverSampler

from sklearn.model_selection import train_test_split
from tensorflow.keras import preprocessing as tfkp
from tensorflow.keras.utils import to_categorical

import gensim.downloader as api

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Ben
[nltk_data]     Lee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import string

def load_data():
    filename = "../data/twitter_data_fixed.pkl"
    print("Loading data from file: " + filename)
    data = pickle.load(open(filename, 'rb'))
    x_text = []
    labels = []
    ids = []
    for i in range(len(data)):
        text = "".join(l for l in data[i]['text'] if l not in string.punctuation)
        x_text.append((data[i]['text']).encode('utf-8'))
        labels.append(data[i]['label'])
    return x_text,labels

In [4]:
def is_url(url):
  try:
    result = urlparse(url)
    return all([result.scheme, result.netloc])
  except ValueError:
    return False

In [5]:
# embedding stuff

def map_weights(embed_dict, vocab, embed_size): # embed size is embedding dim
    vocab_size = len(vocab) + 1
    weights = np.zeros((vocab_size, embed_size))

    n_missed = 0
    words_missed = []
    for k,v in vocab.items():
        try:
            weights[v] = embed_dict[k]  # weights[v] is an index, embed_dict[k] is the list of weights
        except:
            n_missed += 1
            words_missed.append(k)
    print(f"{n_missed} embeddings missed of {vocab_size}")
    return weights, words_missed

### Loading Data

In [6]:
x_text, labels_og = load_data()
labels, uniques = pd.factorize(labels_og)

Loading data from file: ../data/twitter_data_fixed.pkl


### Intitial Twitter-Specific Pre-processing

In [7]:
comments = pd.DataFrame({'comment': x_text, 'attack': labels})

# decode to UTF-8
comments['comment'] = comments['comment'].str.decode("utf-8")

# remove missing rows
comments['comment'].dropna(inplace=True)

# remove usernames
comments['comment'] = comments['comment'].str.replace('(\@\w+.*?)',"", regex=True)

# lower case everything
comments['comment'] = comments['comment'].str.lower()

# remove URLs
comments['comment'] = [' '.join(y for y in x.split() if not is_url(y)) for x in comments['comment']]

# remove stop words
comments['comment'] = comments['comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords.words('english')]))

# traditionally, would also lemmatize but this was not done in the main data

In [15]:
comments.head()

,comment,attack
0,rt : another bloody instant restaurant week ? ...,0
1,video peshmerga decimating isis far interesting.,0
2,oh really ? instant restaurants ? that's shock...,0
3,rt : good weeks #isis. new front opened #sinja...,0
4,rt : don’t need femisnsn men carry heavy thing...,0


### Train-Test Split and Tokenization

In [9]:
X_train_pretoken, X_midway_pretoken, y_train, y_midway = train_test_split(comments['comment'], comments['attack'], random_state = 42, test_size=0.2)

In [10]:
dim = 25
n_classes = len(np.unique(y_train.values))

tokenizer = tfkp.text.Tokenizer(oov_token="<UNK>", filters='!"$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',)
tokenizer.fit_on_texts(X_train_pretoken)

convert = lambda x: tfkp.sequence.pad_sequences(tokenizer.texts_to_sequences(x),
                                                    maxlen=dim,
                                                    padding='post', truncating='post')

X_train = convert(X_train_pretoken)
X_midway = convert(X_midway_pretoken)

In [11]:
X_test, X_val, y_test, y_val = train_test_split(X_midway, y_midway, random_state = 42, test_size=0.5)

### Oversampling

In [16]:
oversample = RandomOverSampler(random_state=42)
X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)

### Reshaping for input

In [17]:
y_train_onehot = to_categorical(y_train, n_classes)
y_val_onehot = to_categorical(y_val, n_classes)
y_test_onehot = to_categorical(y_test, n_classes)
y_train_over_onehot = to_categorical(y_train_over, n_classes)

### Glove Embedding Weights

In [18]:
glove_dict = api.load("glove-twitter-200")
glove_weights, glove_words_missed = map_weights(glove_dict, tokenizer.word_index, 200)

3093 embeddings missed of 15155


### Word2vec Weights

In [19]:
word_dict = api.load("word2vec-google-news-300")
word2vec_weights, word2vec_words_missed = map_weights(word_dict, tokenizer.word_index, 300)

[==================================================] 100.0% 1662.8/1662.8MB downloaded
3546 embeddings missed of 15155


#### Checks

In [20]:
print(np.shape(X_train))
print(np.shape(X_train_over))
print(np.shape(X_val))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_train_over))
print(np.shape(y_val))
print(np.shape(y_test))
print(np.shape(y_train_over_onehot))
print(np.shape(glove_weights))

(12872, 25)
(26508, 25)
(1609, 25)
(1609, 25)
(12872,)
(26508,)
(1609,)
(1609,)
(15155, 200)


In [21]:
from collections import Counter
print(Counter(y_train))
print(Counter(y_train_over))

Counter({0: 8836, 2: 2503, 1: 1533})
Counter({2: 8836, 0: 8836, 1: 8836})


### Write Embedders

In [22]:
from keras.layers import Embedding
from tensorflow.keras import initializers

vocab_size = len(tokenizer.word_index)+1

random_embedder = Embedding(vocab_size, 200, input_length=dim, trainable=True)
glove_embedding = Embedding(vocab_size, 200, embeddings_initializer=initializers.Constant(glove_weights),
                            trainable=False)
word2vec_embedding = Embedding(vocab_size, 300, embeddings_initializer=initializers.Constant(word2vec_weights),
                              trainable=False)

### Load DL Models and Run Them

In [34]:
from DL_models import lstm_keras, cnn_keras, blstm, blstm_atten

In [25]:
lstm_random = lstm_keras(dim,n_classes,random_embedder)
lstm_random_over = lstm_keras(dim,n_classes,random_embedder)
lstm_glove = lstm_keras(dim,n_classes,glove_embedding)
lstm_glove_over = lstm_keras(dim,n_classes,glove_embedding)
lstm_word2vec = lstm_keras(dim,n_classes,word2vec_embedding)
lstm_word2vec_over = lstm_keras(dim,n_classes,word2vec_embedding)

In [128]:
blstm_random = blstm(dim_,n_classes,random_embedder)
blstm_random_over = blstm(dim,n_classes,random_embedder)
blstm_glove = blstm(dim,n_classes,glove_embedding)
blstm_glove_over = blstm(dim,n_classes,glove_embedding)

In [37]:
#BLSTM and CNN need work

blstm_atten_random = blstm_atten(dim,n_classes,random_embedder)
blstm_atten_random_over = blstm_atten(embed_size,n_classes,random_embedder)
blstm_atten_glove = blstm_atten(embed_size,n_classes,glove_embedding)
blstm_atten_glove_over = blstm_atten(embed_size,n_classes,glove_embedding)

StagingError: Exception encountered when calling layer "att_layer_1" (type AttLayer).

in user code:

    File "G:\My Drive\Uni\Experiment Design\Ass2\Deep Learning\DL_models.py", line 70, in call  *
        eij = K.tanh(K.dot(x, self.W))
    File "c:\users\ben lee\appdata\local\programs\python\python39\lib\site-packages\keras\backend.py", line 2125, in dot
        y_permute_dim = [y_permute_dim.pop(-2)] + y_permute_dim

    IndexError: pop index out of range


Call arguments received:
  • x=tf.Tensor(shape=(None, 25, 50), dtype=float32)
  • mask=None

In [35]:
cnn_random = cnn_keras(dim,n_classes,random_embedder)
cnn_random_over = cnn_keras(dim,n_classes,random_embedder)
cnn_glove = cnn_keras(dim,n_classes,glove_embedding)
cnn_glove_over = cnn_keras(dim,n_classes,glove_embedding)

In [36]:
cnn_random.summary()
cnn_random.fit(X_train, y_train_onehot, epochs=3, validation_data=(X_val, y_val_onehot))

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 200)           3031000   
                                                                 
 dropout_37 (Dropout)        (None, 25, 200)           0         
                                                                 
 conv1d_8 (Conv1D)           (None, 21, 128)           128128    
                                                                 
 global_max_pooling1d_8 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_38 (Dropout)        (None, 128)               0         
                                                                 
 dense_18 (Dense)            (None, 3)                 387       
                                                     

InvalidArgumentError:  logits and labels must have the same first dimension, got logits shape [32,3] and labels shape [96]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits
 (defined at c:\users\ben lee\appdata\local\programs\python\python39\lib\site-packages\keras\backend.py:5113)
]] [Op:__inference_train_function_18112]

Errors may have originated from an input operation.
Input Source operations connected to node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits:
In[0] sparse_categorical_crossentropy/Reshape_1 (defined at c:\users\ben lee\appdata\local\programs\python\python39\lib\site-packages\keras\backend.py:5109)	
In[1] sparse_categorical_crossentropy/Reshape (defined at c:\users\ben lee\appdata\local\programs\python\python39\lib\site-packages\keras\backend.py:3561)

Operation defined at: (most recent call last)
>>>   File "c:\users\ben lee\appdata\local\programs\p

In [163]:
lstm_random.summary()
lstm_random.fit(X_train,  y_train_onehot, epochs=3, validation_data=(X_val, y_val_onehot))

Model: "sequential_73"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 25, 200)           3031000   
                                                                 
 dropout_146 (Dropout)       (None, 25, 200)           0         
                                                                 
 lstm_61 (LSTM)              (None, 25)                22600     
                                                                 
 dropout_147 (Dropout)       (None, 25)                0         
                                                                 
 dense_58 (Dense)            (None, 3)                 78        
                                                                 
Total params: 3,053,678
Trainable params: 3,053,678
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
403/403 [========================

In [164]:
lstm_glove.summary()
lstm_glove.fit(X_train,  y_train_onehot, epochs=3, validation_data=(X_val, y_val_onehot))

Model: "sequential_75"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_23 (Embedding)    (None, None, 200)         3031000   
                                                                 
 dropout_150 (Dropout)       (None, None, 200)         0         
                                                                 
 lstm_63 (LSTM)              (None, 25)                22600     
                                                                 
 dropout_151 (Dropout)       (None, 25)                0         
                                                                 
 dense_60 (Dense)            (None, 3)                 78        
                                                                 
Total params: 3,053,678
Trainable params: 22,678
Non-trainable params: 3,031,000
_________________________________________________________________
Epoch 1/3
403/403 [===================

In [26]:
lstm_word2vec.summary()
lstm_word2vec.fit(X_train,  y_train_onehot, epochs=3, validation_data=(X_val, y_val_onehot))

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 300)         4546500   
                                                                 
 dropout_16 (Dropout)        (None, None, 300)         0         
                                                                 
 lstm_8 (LSTM)               (None, 25)                32600     
                                                                 
 dropout_17 (Dropout)        (None, 25)                0         
                                                                 
 dense_8 (Dense)             (None, 3)                 78        
                                                                 
Total params: 4,579,178
Trainable params: 32,678
Non-trainable params: 4,546,500
_________________________________________________________________
Epoch 1/3
403/403 [====================

In [137]:
blstm_random_over.summary()
blstm_random_over.fit(X_train_over,  y_train_over_onehot, epochs=3, validation_data=(X_val, y_val_onehot))

Model: "sequential_66"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 25, 25)            378875    
                                                                 
 dropout_132 (Dropout)       (None, 25, 25)            0         
                                                                 
 bidirectional_7 (Bidirectio  (None, 50)               10200     
 nal)                                                            
                                                                 
 dropout_133 (Dropout)       (None, 50)                0         
                                                                 
 dense_53 (Dense)            (None, 3)                 153       
                                                                 
Total params: 389,228
Trainable params: 389,228
Non-trainable params: 0
_______________________________________________